In [3]:
import numpy as np

# 定义微分方程 dy/dx = -x^2 y^2
def f(x, y):
    return -x**2 * y**2

# 定义精确解 y(x) = 3/(1 + x^3)
def exact_solution(x):
    return 3.0 / (1.0 + x**3)

# 四阶 Runge-Kutta 方法
def runge_kutta_4th_order(f, x0, y0, h, x_end):
    x = x0
    y = y0
    while x < x_end:
        if x + h > x_end:
            h = x_end - x
        k1 = h * f(x, y)
        k2 = h * f(x + h/2, y + k1/2)
        k3 = h * f(x + h/2, y + k2/2)
        k4 = h * f(x + h, y + k3)
        y += (k1 + 2*k2 + 2*k3 + k4) / 6
        x += h
    return y

# 四阶显式 Adams-Bashforth 方法（预估步）
def adams_bashforth_4th_order(f, x, y, h):
    return y[-1] + (h/24)*(55*f(x[-1], y[-1]) - 59*f(x[-2], y[-2]) + 37*f(x[-3], y[-3]) - 9*f(x[-4], y[-4]))

# 四阶隐式 Adams-Moulton 方法（校正步）
def adams_moulton_4th_order(f, x, y, h, y_pred):
    # 使用牛顿迭代法求解隐式方程
    def implicit_eq(y_new):
        return y_new - y[-1] - (h/24)*(9*f(x[-1] + h, y_new) + 19*f(x[-1], y[-1]) - 5*f(x[-2], y[-2]) + f(x[-3], y[-3]))
    
    y_new = y_pred  # 初始猜测为预估值
    for _ in range(10):  # 进行几次迭代
        y_new = y_new - implicit_eq(y_new) / (1 - (9*h/24)*(- (x[-1] + h)**2 * 2 * y_new))  # 牛顿迭代
    return y_new

# 四阶隐式 Adams 方法（预估-校正）
def adams_implicit_4th_order(f, x0, y0, h, x_end):
    x = [x0]
    y = [y0]
    # 使用 RK4 计算前三个点
    for i in range(3):
        y_next = runge_kutta_4th_order(f, x[-1], y[-1], h, x[-1] + h)
        x.append(x[-1] + h)
        y.append(y_next)
    # 使用预估-校正法计算后续点
    while x[-1] < x_end:
        if x[-1] + h > x_end:
            h = x_end - x[-1]
        y_pred = adams_bashforth_4th_order(f, x, y, h)
        y_next = adams_moulton_4th_order(f, x, y, h, y_pred)
        x.append(x[-1] + h)
        y.append(y_next)
    return y[-1]

# 主函数
def main():
    x0 = 0.0
    y0 = 3.0
    x_end = 1.5
    exact_y = exact_solution(x_end)
    
    results = []
    for l in range(4):
        h = 0.1 / (2**l)
        # 使用 RK4 计算
        rk4_y = runge_kutta_4th_order(f, x0, y0, h, x_end)
        rk4_error = abs(rk4_y - exact_y)
        
        # 使用隐式 Adams 方法计算
        adams_y = adams_implicit_4th_order(f, x0, y0, h, x_end)
        adams_error = abs(adams_y - exact_y)
        
        results.append((h, rk4_error, adams_error))
    
    # 计算误差阶
    print("四阶 Runge-Kutta 方法误差阶：")
    for i in range(1, 4):
        o_rk4 = np.log(results[i-1][1]/results[i][1])/np.log(2)
        print(f"h = {results[i][0]:.4f}, 误差阶: {o_rk4:.2f}")
        print(f"error={results[i][1]:.15f}")
    
    print("\n四阶隐式 Adams 方法误差阶：")
    for i in range(1, 4):
        o_adams = np.log(results[i-1][2]/results[i][2])/np.log(2)
        print(f"h = {results[i][0]:.4f}, 误差阶: {o_adams:.2f}")
        print(f"error={results[i][2]:.15f}")
if __name__ == "__main__":
    main()

四阶 Runge-Kutta 方法误差阶：
h = 0.0500, 误差阶: 4.09
error=0.000001042240727
h = 0.0250, 误差阶: 4.05
error=0.000000062907181
h = 0.0125, 误差阶: 4.03
error=0.000000003861850

四阶隐式 Adams 方法误差阶：
h = 0.0500, 误差阶: 3.76
error=0.000002121451304
h = 0.0250, 误差阶: 3.96
error=0.000000136240184
h = 0.0125, 误差阶: 4.00
error=0.000000008522090
